# Week 1 Lab: Data Normalization

In this lab, you will learn how to transform a denormalized table or "One Big Table" (OBT) into First Normal Form (1NF), Second Normal Form (2NF), and Third Normal Form (3NF). This process is fundamental to database normalization, which helps reduce data redundancy and improve data integrity.

To open the solution notebook, follow these steps:
- Go to the main menu and select `File -> Preferences -> Settings`.
- Click on `Text Editor` on the left, then scroll down to the `Files: Exclude` section.
- Remove the line `**/C4_W1_Lab_1_Data_Normalization_Solution.ipynb`. The file will now appear in the explorer.
- You can close the `Settings` tab.

# Table of Contents

- [ 1 - Introduction](#1)
  - [ 1.1 - Data Normalization](#1-1)
  - [ 1.2 - Dataset](#1-2)
- [ 2 - First Normal Form (1NF)](#2)
- [ 3 - Second Normal Form (2NF)](#3)
- [ 4 - Third Normal Form (3NF)](#4)

<a id='1'></a>
## 1 - Introduction

As a Data Engineer, you may not frequently perform data normalization from scratch, but understanding this process is crucial. Typically, you will encounter source databases that are already normalized, and your task will often involve denormalizing this data to make it useful for extracting analytical insights or solving business questions. This lab focuses on the opposite process: taking a dataset stored as a One Big Table and normalizing it up to the third normal form, which is common in transactional systems.

<a id='1-1'></a>
### 1.1 - Data Normalization

Normalization is a database design technique that involves dividing large tables into smaller, less redundant tables and defining relationships between them. The goal is to isolate data so that additions, deletions, and modifications can be made in a single table and then propagated through the rest of the database using defined relationships.

Here are some of the normalization benefits: 
- Reduce Data Redundancy: Eliminating duplicate data saves storage space and ensures consistency across the database.
- Improve Data Integrity: Ensuring that each piece of data is stored in only one place reduces the likelihood of data anomalies and maintains the accuracy of the data.
- Enhance Update/Delete Query Performance

<a id='1-2'></a>
### 1.2 - Dataset

The dataset you'll use contains similar information to the normalized classicmodels you used in previous labs. However, in this lab, the data is stored as One Big Table, originally extracted from a multiline JSON file, where each JSON object has the following structure:

```json
{
  "orderNumber": 10100,
  "orderDate": "2003-01-06",
  "requiredDate": "2003-01-13",
  "shippedDate": "2003-01-10",
  "status": "Shipped",
  "comments": null,
  "orderDetails": [
    {
      "productCode": "S18_1749",
      "quantityOrdered": 30,
      "priceEach": 136.00
    },
    {
      "productCode": "S18_2248",
      "quantityOrdered": 50,
      "priceEach": 55.09
    },
    {
      "productCode": "S18_4409",
      "quantityOrdered": 22,
      "priceEach": 75.46
    },
    {
      "productCode": "S24_3969",
      "quantityOrdered": 49,
      "priceEach": 35.29
    }
  ],
  "customer": {
    "customerName": "Online Diecast Creations Co.",
    "contactLastName": "Young",
    "contactFirstName": "Dorothy",
    "phone": "6035558647",
    "addressLine1": "2304 Long Airport Avenue",
    "addressLine2": null,
    "city": "Nashua",
    "state": "NH",
    "postalCode": "62005",
    "country": "USA",
    "salesRepEmployeeNumber": 1216.00,
    "creditLimit": 114200.00
  }
}
```

This dataset has already been uploaded into the database as a table named `orders` under the schema `classicmodels_obt`. Here's the table's schema:


<img src="./images/ERD_OBT.png" width="300">

Note that the `orderDetails` and `customer` fields are saved as JSON objects directly in the database: each object is structured as a dictionary with key-value pairs holding information about each order and the customer that placed that order.

To explore the data, let's import all the necessary packages and SQL extensions for running the `%sql` magic commands used in this notebook:

In [1]:
import os 
import json

import pandas as pd
import psycopg2

from IPython.display import HTML
from dotenv import load_dotenv
from sqlalchemy import create_engine

pd.set_option('display.max_columns', 30)

In [2]:
%load_ext sql

Run the following code to get the link to the AWS console.

*Note*: For security reasons, the URL to access the AWS console will expire every 15 minutes, but any AWS resources you created will remain available for the 2 hour period. If you need to access the console after 15 minutes, please rerun this code cell to obtain a new active link.

In [ ]:
with open('../.aws/aws_console_url', 'r') as file:
    aws_url = file.read().strip()

HTML(f'<a href="{aws_url}" target="_blank">GO TO AWS CONSOLE</a>')

*Note:* If you see the window like in the following printscreen, click on **logout** link, close the window and click on console link again.

![AWSLogout](images/AWSLogout.png)

In the AWS console, go to **CloudFormation**.
- Click on the alphanumeric stack name and find the **Outputs** tab. You will see the key `PostgresEndpoint` and its corresponding **Value** column. Copy the value.
- Edit the `./src/env` file, replacing the placeholder `<RDS-ENDPOINT>` with the endpoint value. 
- Save changes to the file.

Execute the following cell to load the environment variables and connect to the database:

In [3]:
load_dotenv('./src/env', override=True)

DBHOST = os.getenv('DBHOST')
DBPORT = os.getenv('DBPORT')
DBNAME = os.getenv('DBNAME')
DBUSER = os.getenv('DBUSER')
DBPASSWORD = os.getenv('DBPASSWORD')

connection_url = f"postgresql+psycopg2://{DBUSER}:{DBPASSWORD}@{DBHOST}:{DBPORT}/{DBNAME}"

%sql {connection_url}
     

Explore the loaded data:

In [ ]:
%%sql
select count(*) from classicmodels_obt.orders;

In [ ]:
%%sql
select * from classicmodels_obt.orders limit 3;

The `orderdetails` and `customer` fields are not of basic data types, they are dictionaries with the following structure:

- `orderdetails` is a list of dictionaries/JSON objects about all the products placed within an individual order. Each entry in the list refers to a product, with keys on its product code, the quantity ordered and the unitary price.

- `customer` field contains a dictionary/JSON object where each key corresponds to a feature of the customer that placed the order, such as personal details, contact, and location.

Now you will convert this denormalized table to a first normal form, by unnesting these two columns and ensuring that the table has a unique primary key.

<a id='2'></a>
## 2 - First Normal Form (1NF)

Your first task is to create the First Normal Form (1NF) and insert it into the database. For that, you will read the data using an SQL query and then transform the data using `pandas` package. 

The features of the First Normal Form is that each column has a single value, meaning there should be no nested data in any columns, and that the table has a unique primary key:  

- For that, you will unnest each list in the `orderdetails` column, meaning you will create a row for each element in the list. In addition, since each element is a JSON object or dictionary, you will  unnest each element by creating a new field or column for each key inside this dictionary. You will repeat this last step for the `customer` field.

- After you unnest these two columns, each row will represent an item placed within an order. So to uniquely identify each row, you will create a column `orderlinenumber`that denotes the item's number within its corresponding order. The two columns `ordernumber` and `orderlinenumber` will represent the composite primary key of the table. 

The final schema after the 1NF will look like the following image: 

<img src="./images/ERD_1NF_Orders.png" width="400">

2.1. Create the schema in which the relations in 1NF will be stored and then read the OBT dataset and save it as a `pandas` DataFrame.

In [ ]:
%%sql
CREATE SCHEMA IF NOT EXISTS classicmodels_1nf;

Have a look again at the original data:

In [ ]:
result = %sql select * from classicmodels_obt.orders

df = result.DataFrame()

df.head()

2.2. Create a new flat table by unnesting the content of the  `customer` field of the `df` DataFrame. For that, you can use the `pandas` [`json_normalize()` method](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html) over the `customer` field. This function will return a DataFrame with the keys of the key-value pairs as columns in the new table.

*Note*: In the cells where you see the comments `### START CODE HERE ###` and `### END CODE HERE ###` you need to complete the code replacing all `None`. The rest of the cells are already complete, you just need to review and run the code.

In [ ]:
### START CODE HERE ### (1 line of code)
customers_df = pd.json_normalize(df['customer']) # @REPLACE EQUALS pd.None(None['None'])
### END CODE HERE ###

customers_df.head()

##### __Expected Output__

| **city**    | **phone**         | **state** | **country** | **postalCode** | **creditLimit** | **addressLine1**           | **addressLine2** | **customerName**              | **contactLastName** | **contactFirstName** | **salesRepEmployeeNumber** |
| ----------- | ----------------- | --------- | ----------- | -------------- | --------------- | -------------------------- | ---------------- | ----------------------------- | ------------------- | ------------------- | ------------------------ |
| Nashua      | 6035558647        | NH        | USA         | 62005          | 114200.0        | 2304 Long Airport Avenue   | None             | Online Diecast Creations Co.  | Young               | Dorothy              | 1216.0                   |
| Frankfurt   | +49 69 66 90 2555 | None      | Germany     | 60528          | 59700.0         | Lyonerstr. 34              | None             | Blauer See Auto, Co.          | Keitel              | Roland               | 1504.0                   |
| NYC         | 2125551500        | NY        | USA         | 10022          | 76400.0         | 2678 Kingston Rd.          | Suite 101        | Vitachrome Inc.               | Frick               | Michael              | 1286.0                   |
| Stavern     | 07-98 9555        | None      | Norway      | 4110           | 81700.0         | Erling Skakkes gate 78     | None             | Baane Mini Imports            | Bergulfsen          | Jonas                | 1504.0                   |
| Madrid      | (91) 555 94 44    | None      | Spain       | 28034          | 227600.0        | C/ Moralzarzal, 86         | None             | Euro+ Shopping Channel        | Freyre              | Diego                | 1370.0                  |


`pd.json_normalize` creates a DataFrame where each dictionary in the `customer` column is flattened into a row. The index of the original DataFrame is preserved, which is crucial for maintaining the correct relationship between the original rows and the new flattened rows. It is time to concatenate the two datasets. 

2.3. You need to drop the `customer` column from the original dataframe, using the `drop()` method and specifying the column. You should keep `inplace` argument equal to `True`. 

Then concatenate the dataframe `df` with `customers_df`. For that, you will use `pd.concat` method with the `axis` parameter set as 1. Using `pd.concat` with `axis=1` joins the DataFrames column-wise, aligning rows by their index. Since the index is preserved, each row in the flattened `customers_df` dataframe aligns correctly with its corresponding row in the original dataframe.

In [ ]:
### START CODE HERE ### (2 lines of code)
df.drop(columns='customer', inplace=True) # @REPLACE df.None(columns='None', inplace=True)
df = pd.concat([df, customers_df], axis=1) # @REPLACE EQUALS pd.None([df, None], axis=1)
### END CODE HERE ###

df.head()

##### __Expected Output__

*Note*: Some text is omitted.

| **ordernumber** | **orderdate** | **requireddate** | **shippeddate** | **status** | **comments**             | **orderdetails** | **city**    | **phone**         | **state** | **country** | **postalCode** | **creditLimit** | **addressLine1**           | **addressLine2** | **customerName**              | **contactLastName** | **contactFirstName** | **salesRepEmployeeNumber** |
| --------------- | ------------- | ---------------- | --------------- | ---------- | ----------------------- | ---------------- | ----------- | ----------------- | --------- | ----------- | -------------- | --------------- | -------------------------- | ---------------- | ----------------------------- | ------------------- | ------------------- | ------------------------ |
| 10100           | 2003-01-06    | 2003-01-13       | 2003-01-10      | Shipped    | None                    | [{'priceEach': 136.0, 'productCode': 'S18_1749...    | Nashua      | 6035558647        | NH        | USA         | 62005          | 114200.0        | 2304 Long Airport Avenue   | None             | Online Diecast Creations Co.  | Young               | Dorothy              | 1216.0                   |
| 10101           | 2003-01-09    | 2003-01-18       | 2003-01-11      | Shipped    | Check on availability.  | [{'priceEach': 108.06, 'productCode': 'S18_232...    | Frankfurt   | +49 69 66 90 2555 | None      | Germany     | 60528          | 59700.0         | Lyonerstr. 34              | None             | Blauer See Auto, Co.          | Keitel              | Roland               | 1504.0                   |
| 10102           | 2003-01-10    | 2003-01-18       | 2003-01-14      | Shipped    | None                    | [{'priceEach': 95.55, 'productCode': 'S18_1342...    | NYC         | 2125551500        | NY        | USA         | 10022          | 76400.0         | 2678 Kingston Rd.          | Suite 101        | Vitachrome Inc.               | Frick               | Michael              | 1286.0                   |
| 10103           | 2003-01-29    | 2003-02-07       | 2003-02-02      | Shipped    | None                    | [{'priceEach': 214.3, 'productCode': 'S10_1949...    | Stavern     | 07-98 9555        | None      | Norway      | 4110           | 81700.0         | Erling Skakkes gate 78     | None             | Baane Mini Imports            | Bergulfsen          | Jonas                | 1504.0                   |
| 10104           | 2003-01-31    | 2003-02-09       | 2003-02-01      | Shipped    | None                    | [{'priceEach': 131.44, 'productCode': 'S12_314...    | Madrid      | (91) 555 94 44    | None      | Spain       | 28034          | 227600.0        | C/ Moralzarzal, 86         | None             | Euro+ Shopping Channel        | Freyre              | Diego                | 1370.0                  |


2.4. The field `customerName` uniquely identifies each customer. To make it easy to identify each customer, run the following cell to add a numerical ID column with each entry associated with the customer's name. (The code below is complete; no change is required).

In [ ]:
df['customerNumber'] = pd.factorize(df['customerName'])[0] + 1
df.head()

Now that the `customer` field has been transformed into atomic-valued fields, you need to do the same for the `orderdetails` column. 

2.5. Create a new row for each product in the same order using the `pandas` [`explode()` method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html) on `orderdetails` column of the original `df` DataFrame. Make sure to set the `ignore_index` parameter to `True`.

In [ ]:
### START CODE HERE ### (1 line of code)
df_exploded = df.explode('orderdetails', ignore_index=True) # @REPLACE EQUALS df.None('None', ignore_index=None)
### END CODE HERE ###

df_exploded.head()

##### __Expected Output__
*Note*: Some text is omitted.

| **ordernumber** | **orderdate** | **requireddate** | **shippeddate** | **status** | **comments**            | **orderdetails** | **city**    | **phone**         | **state** | **country** | **postalCode** | **creditLimit** | **addressLine1**           | **addressLine2** | **customerName**              | **contactLastName** | **contactFirstName** | **salesRepEmployeeNumber** | **customerNumber** |
| --------------- | ------------- | ---------------- | --------------- | ---------- | ---------------------- | ---------------- | ----------- | ----------------- | --------- | ----------- | -------------- | --------------- | -------------------------- | ---------------- | ----------------------------- | ------------------- | ------------------- | ------------------------ | ------------------ |
| 10100           | 2003-01-06    | 2003-01-13       | 2003-01-10      | Shipped    | None                   | {'priceEach': 136.0, 'productCode': 'S18_1749'...       | Nashua      | 6035558647        | NH        | USA         | 62005          | 114200.0        | 2304 Long Airport Avenue   | None             | Online Diecast Creations Co.  | Young               | Dorothy              | 1216.0                   | 1                  |
| 10100           | 2003-01-06    | 2003-01-13       | 2003-01-10      | Shipped    | None                   |{'priceEach': 55.09, 'productCode': 'S18_2248'...        | Nashua      | 6035558647        | NH        | USA         | 62005          | 114200.0        | 2304 Long Airport Avenue   | None             | Online Diecast Creations Co.  | Young               | Dorothy              | 1216.0                   | 1                  |
| 10100           | 2003-01-06    | 2003-01-13       | 2003-01-10      | Shipped    | None                   | {'priceEach': 75.46, 'productCode': 'S18_4409'...       | Nashua      | 6035558647        | NH        | USA         | 62005          | 114200.0        | 2304 Long Airport Avenue   | None             | Online Diecast Creations Co.  | Young               | Dorothy              | 1216.0                   | 1                  |
| 10100           | 2003-01-06    | 2003-01-13       | 2003-01-10      | Shipped    | None                   | {'priceEach': 35.29, 'productCode': 'S24_3969'...          | Nashua      | 6035558647        | NH        | USA         | 62005          | 114200.0        | 2304 Long Airport Avenue   | None             | Online Diecast Creations Co.  | Young               | Dorothy              | 1216.0                   | 1                  |
| 10101           | 2003-01-09    | 2003-01-18       | 2003-01-11      | Shipped    | Check on availability. | {'priceEach': 108.06, 'productCode': 'S18_2325'...          | Frankfurt   | +49 69 66 90 2555 | None      | Germany     | 60528          | 59700.0         | Lyonerstr. 34              | None             | Blauer See Auto, Co.          | Keitel              | Roland               | 1504.0                   | 2                  |


The `explode` function is used to transform each element of a list-like column into a separate row. Setting `ignore_index=True` resets the index of the resulting dataframe, creating a new integer index that starts from 0. 

2.6. Use again the `json_normalize` over the `orderdetails` column of the dataframe `df_exploded`.

In [ ]:
### START CODE HERE ### (1 line of code)
orderdetails_normalized = pd.json_normalize(df_exploded['orderdetails']) # @REPLACE EQUALS pd.None(None['None'])
### END CODE HERE ###

orderdetails_normalized.head()

##### __Expected Output__

| **priceEach** | **productCode** | **quantityOrdered** |
| ------------- | --------------- | ------------------- |
| 136.00        | S18_1749        | 30                  |
| 55.09         | S18_2248        | 50                  |
| 75.46         | S18_4409        | 22                  |
| 35.29         | S24_3969        | 49                  |
| 108.06        | S18_2325        | 25                 |


2.7. Finally, `drop()` the `orderdetails` column from the original `df_exploded` DataFrame keeping `inplace` argument equal to `True`.  Then `concat()` `df_exploded` dataframe with the `orderdetails_normalized` DataFrame.

In [ ]:
### START CODE HERE ### (2 lines of code)
df_exploded.drop(columns='orderdetails', inplace=True) # @REPLACE df_exploded.None(columns='None', inplace=True)
df_normalized = pd.concat([df_exploded, orderdetails_normalized], axis=1) # @REPLACE EQUALS pd.None([df_exploded, None], axis=1)
### END CODE HERE ###

# First Normal Form
df_normalized.head()

##### __Expected Output__

| **ordernumber** | **orderdate** | **requireddate** | **shippeddate** | **status** | **comments**            | **city**    | **phone**         | **state** | **country** | **postalCode** | **creditLimit** | **addressLine1**           | **addressLine2** | **customerName**              | **contactLastName** | **contactFirstName** | **salesRepEmployeeNumber** | **customerNumber** | **priceEach** | **productCode** | **quantityOrdered** |
| --------------- | ------------- | ---------------- | --------------- | ---------- | ---------------------- | ----------- | ----------------- | --------- | ----------- | -------------- | --------------- | -------------------------- | ---------------- | ----------------------------- | ------------------- | ------------------- | ------------------------ | ------------------ | ------------ | --------------- | ------------------- |
| 10100           | 2003-01-06    | 2003-01-13       | 2003-01-10      | Shipped    | None                   | Nashua      | 6035558647        | NH        | USA         | 62005          | 114200.0        | 2304 Long Airport Avenue   | None             | Online Diecast Creations Co.  | Young               | Dorothy              | 1216.0                   | 1                  | 136.00        | S18_1749        | 30                  |
| 10100           | 2003-01-06    | 2003-01-13       | 2003-01-10      | Shipped    | None                   | Nashua      | 6035558647        | NH        | USA         | 62005          | 114200.0        | 2304 Long Airport Avenue   | None             | Online Diecast Creations Co.  | Young               | Dorothy              | 1216.0                   | 1                  | 55.09         | S18_2248        | 50                  |
| 10100           | 2003-01-06    | 2003-01-13       | 2003-01-10      | Shipped    | None                   | Nashua      | 6035558647        | NH        | USA         | 62005          | 114200.0        | 2304 Long Airport Avenue   | None             | Online Diecast Creations Co.  | Young               | Dorothy              | 1216.0                   | 1                  | 75.46         | S18_4409        | 22                  |
| 10100           | 2003-01-06    | 2003-01-13       | 2003-01-10      | Shipped    | None                   | Nashua      | 6035558647        | NH        | USA         | 62005          | 114200.0        | 2304 Long Airport Avenue   | None             | Online Diecast Creations Co.  | Young               | Dorothy              | 1216.0                   | 1                  | 35.29         | S24_3969        | 49                  |
| 10101           | 2003-01-09    | 2003-01-18       | 2003-01-11      | Shipped    | Check on availability. | Frankfurt   | +49 69 66 90 2555 | None      | Germany     | 60528          | 59700.0         | Lyonerstr. 34              | None             | Blauer See Auto, Co.          | Keitel              | Roland               | 1504.0                   | 2                  | 108.06        | S18_2325        | 25                  |


The resulting DataFrame `df_normalized` should be in the Frist Normal Form now.

2.8. Now that you have atomic values in your dataset, each row represents a product placed within an order. So to uniquely identify each row, you need to have a composite primary key which consists of two columns: the first column represents the order id which is already provided as `ordernumber`. The second column should identify the product within an order. So let's create an additional column to identify each product in each order. You will call it Order Line Number (`orderlinenumber`).

In [ ]:
df_normalized['orderlinenumber'] = df_normalized.groupby('ordernumber').cumcount() + 1
df_normalized.head()

2.9. With these transformations, you have finished the normalization process up to 1NF. Let's insert this dataset into your database. Run the following cell to drop the table if it has been loaded before to avoid an error.

In [ ]:
%%sql
DROP TABLE IF EXISTS classicmodels_1nf.orders

Populate the table `orders` under the schema `classicmodels_1nf` with the data from the DataFrame `df_normalized`.

In [ ]:
engine = create_engine(connection_url)

df_normalized.to_sql('orders', engine, schema='classicmodels_1nf', index=False)

Inspect the data that you just loaded.

In [ ]:
%%sql 
SELECT COUNT(*) FROM classicmodels_1nf.orders;

In [ ]:
%%sql 
SELECT * FROM classicmodels_1nf.orders LIMIT 10;

<a id='3'></a>
## 3 - Second Normal Form (2NF)

For the second normal form (2NF), the requirements of first normal form must be met and any partial dependencies should be removed. A partial dependency occurs when there is a subset of non-key columns that depend on some columns in the composite key. 

 In the table of the first normal form, the columns that contain information related to the order such as `orderDate`, `shippedDate`, `status` and `comments` only depend on the `ordernumber` column of the composite primary key. Moreover all the columns that contain information about the customers (`city`, `phone`, `state`, `country`, `postalCode`, `creditLimit`, `addressLine1`, `addressLine2`, `customerName`, `customerLastName`, `customerFirstName`, `salesRepEmployeeNumber`, `customerNumber`) also depend only on the `ordernumber` column. The remaining columns`priceEach`, `productCode`, `quantityOrdered` require the two columns of the composite primary key. So, to move from 1NF to 2NF, you will split the table of the first normal form into two tables:
- `orderdetails` that contains the details of each item placed within each order;
- `orders` that contains information related to the orders, meaning that could be uniquely identified using the ordernumber column.

The final schema after the 2NF transformations will look like this:

<img src="./images/ERD_2NF.png" width="400">

3.1. Create the `classicmodels_2nf` schema and then read the 1NF dataset and transform it to create the 2NF version.

In [ ]:
%%sql
CREATE SCHEMA IF NOT EXISTS classicmodels_2nf;

Have a look again at the data in the 1NF:

In [ ]:
result = %sql select * from classicmodels_1nf.orders
df_orders = result.DataFrame()

df_orders.head()

3.2. Extract all the information related to `orderdetails` and create a table with the values for the order details. 
- Take only the orderdetails related columns from your `df_orders` dataframe (see the list in the cell below) and make a copy of it with the `copy()` method naming it `df_orderdetails`.

In [ ]:
orderdetails_columns = ['ordernumber',
                        'orderlinenumber',
                        'priceEach',
                        'productCode',
                        'quantityOrdered'
                       ]

### START CODE HERE ### (1 line of code)
df_orderdetails = df_orders[orderdetails_columns].copy() # @REPLACE EQUALS None[None].None()
### END CODE HERE ###

df_orderdetails.head()

##### __Expected Output__


| **ordernumber** | **orderlinenumber**  |**priceEach** | **productCode** | **quantityOrdered** | 
| --------------- | ---------------------| -------------| ----------------| --------------------|
| 10100           | 1                    | 136.00       |S18_1749         | 30                  | 
| 10100           | 2                    | 55.09        |S18_2248         | 50                  | 
| 10100           | 3                    | 75.46        |S18_4409         | 22                  | 
| 10100           | 4                    | 35.29        |S24_3969         | 49                  | 
| 10101           | 1                    | 108.06       |S18_2325         | 25                  |


3.3. Now that you have the orderdetails stored in a separate table, that information can be dropped from the original DataFrame. The only necessary column to keep is the `ordernumber` as it helps relate the orders with orderdetails' information. Run the following cell to create a list of the columns you need to drop from the `df_orders` DataFrame:

In [ ]:
orderdetails_columns.pop(0)
orderdetails_columns

3.4. Drop the `orderdetails_columns` from the DataFrame `df_orders`. The `inplace` argument should be equal to `True`. After that,  drop the duplicate rows from `df_orders` using the method `drop_duplicates`. Make sure that you put argument `inplace` equal to `True`.

In [ ]:
### START CODE HERE ### (2 lines of code)
df_orders.drop(columns=orderdetails_columns, inplace=True) # @REPLACE None.None(columns=None, inplace=None)
df_orders.drop_duplicates(inplace=True) # @REPLACE None.None(inplace=None)
### END CODE HERE ###

df_orders.head()

##### __Expected Output__

| **ordernumber** | **orderdate** | **requireddate** | **shippeddate** | **status** | **comments** | **city** | **phone** | **state** | **country** | **postalCode** | **creditLimit** | **addressLine1** | **addressLine2** | **customerName** | **contactLastName** | **contactFirstName** | **salesRepEmployeeNumber** | **customerNumber** |
|----------------:|:--------------|:-----------------|:----------------|:-----------|:-------------|:---------|:----------|:----------|:------------|---------------:|----------------:|:-----------------|:-----------------|:-----------------|:--------------------|:---------------------|---------------------------:|-------------------:|
|           10100 | 2003-01-06    | 2003-01-13       | 2003-01-10      | Shipped    | None         | Nashua   | 6035558647 | NH       | USA         |          62005 |         114,200 | 2304 Long Airport Avenue | None | Online Diecast Creations Co. | Young | Dorothy | 1216 | 1 |
|           10101 | 2003-01-09    | 2003-01-18       | 2003-01-11      | Shipped    | Check on availability. | Frankfurt | +49 69 66 90 2555 | None | Germany | 60528 | 59,700 | Lyonerstr. 34 | None | Blauer See Auto, Co. | Keitel | Roland | 1504 | 2 |
|           10102 | 2003-01-10    | 2003-01-18       | 2003-01-14      | Shipped    | None         | NYC      | 2125551500 | NY       | USA         |          10022 |          76,400 | 2678 Kingston Rd. | Suite 101 | Vitachrome Inc. | Frick | Michael | 1286 | 3 |
|           10103 | 2003-01-29    | 2003-02-07       | 2003-02-02      | Shipped    | None         | Stavern  | 07-98 9555 | None     | Norway      |           4110 |          81,700 | Erling Skakkes gate 78 | None | Baane Mini Imports | Bergulfsen | Jonas | 1504 | 4 |
|           10104 | 2003-01-31    | 2003-02-09       | 2003-02-01      | Shipped    | None         | Madrid   | (91) 555 94 44 | None | Spain | 28034 | 227,600 | C/ Moralzarzal, 86 | None | Euro+ Shopping Channel | Freyre | Diego | 1370 | 5 |


3.5. The two tables you just created can be related through the `ordernumber` field. Insert the two tables into the `classicmodels_2nf` schema. You will need to drop the tables before that in case they have been added before.

In [ ]:
%%sql
DROP TABLE IF EXISTS classicmodels_2nf.orders

In [ ]:
%%sql
DROP TABLE IF EXISTS classicmodels_2nf.orderdetails

In [ ]:
df_orders.to_sql('orders', engine, schema='classicmodels_2nf', index=False)

In [ ]:
df_orderdetails.to_sql('orderdetails', engine, schema='classicmodels_2nf', index=False)

Explore the tables.

In [ ]:
%%sql
SELECT COUNT(*) FROM classicmodels_2nf.orders;

In [ ]:
%%sql
SELECT COUNT(*) FROM classicmodels_2nf.orderdetails;

In [ ]:
%%sql
SELECT * FROM classicmodels_2nf.orderdetails limit 10;

In [ ]:
%%sql
SELECT * FROM classicmodels_2nf.orders limit 10;

<a id='4'></a>
## 4 - Third Normal Form (3NF)

The features of 3NF are the following:
- Already in 2NF: The table must already be in Second Normal Form (2NF).
- No Transitive Dependencies: There should be no transitive dependencies. A transitive dependency occurs when a non-key column depends on another non-key column. 

In the `orders` table, you have the following columns: `phone`, `state`, `country`, `postalCode`, `creditLimit`, `addressLine1`, `addressLine2`, `customerName`, `customerLastName`, `customerFirstName`, and `salesRepEmployeeNumber`. All of these columns are non-key columns that depends on the non-key column `customerNumber`. 

So to transform the 2NF form into 3NF form, you need to remove these columns from the `orders` table and create a new table that contains the details of each customer.

The final schema of the 3NF normalization is the following:


<img src="./images/ERD_3NF.png" width="600">

4.1. Create the `classicmodels_3nf` schema to store your transformed tables there.

In [ ]:
%%sql
CREATE SCHEMA IF NOT EXISTS classicmodels_3nf;

4.2. You will need to read the `orders` and `orderdetails` tables from the `classicmodels_2nf` schema. Although you will not make any further changes to the `orderdetails` table in this step, you will upload it into the 3NF schema to keep all your datasets in the same place.

Read the `orders` table into the `df_orders` pandas DataFrame:

In [ ]:
result = %sql select * from classicmodels_2nf.orders
df_orders = result.DataFrame()

df_orders.head()

Read the `orderdetails` table into `df_orderdetails`:

In [ ]:
result = %sql select * from classicmodels_2nf.orderdetails
df_orderdetails = result.DataFrame()

df_orderdetails.head()

4.3. To create the customers table, extract only the customer-related columns from your `df_orders` DataFrame (see the list in the cell below) and make a copy of it with the `copy()` method naming it `df_customers`. After that drop the duplicate rows from `df_customers` using the method `drop_duplicates`. Make sure that you put argument `inplace` equal to `True`.


In [ ]:
customer_columns = ['customerNumber', 
                    'customerName', 
                    'contactLastName', 
                    'contactFirstName', 
                    'phone', 
                    'addressLine1', 
                    'addressLine2',
                    'postalCode',                     
                    'city', 
                    'state', 
                    'country', 
                    'creditLimit',
                    'salesRepEmployeeNumber'
                   ] 

### START CODE HERE ### (2 lines of code)
df_customers = df_orders[customer_columns].copy() # @REPLACE EQUALS None[None].None()
df_customers.drop_duplicates(inplace=True) # @REPLACE None.None(inplace=None)
### END CODE HERE ###

df_customers.head()

##### __Expected Output__

| **customerNumber** | **customerName**                | **contactLastName** | **contactFirstName** | **phone**         | **addressLine1**           | **addressLine2** | **postalCode** | **city**  | **state** | **country** | **creditLimit** | **salesRepEmployeeNumber** |
| ------------------ | -------------------------------- | ------------------- | ------------------- | ----------------- | -------------------------- | ---------------- | -------------- | -------- | --------- | ----------- | --------------- | ------------------------- |
| 1                  | Online Diecast Creations Co.   | Young               | Dorothy             | 6035558647        | 2304 Long Airport Avenue   | None             | 62005          | Nashua   | NH        | USA         | 114200.0        | 1216.0                    |
| 2                  | Blauer See Auto, Co.            | Keitel              | Roland              | +49 69 66 90 2555 | Lyonerstr. 34              | None             | 60528          | Frankfurt | None      | Germany     | 59700.0         | 1504.0                    |
| 3                  | Vitachrome Inc.                 | Frick                | Michael             | 2125551500        | 2678 Kingston Rd.         | Suite 101        | 10022          | NYC      | NY        | USA         | 76400.0         | 1286.0                    |
| 4                  | Baane Mini Imports              | Bergulfsen           | Jonas               | 07-98 9555        | Erling Skakkes gate 78    | None             | 4110           | Stavern  | None      | Norway      | 81700.0         | 1504.0                    |
| 5                  | Euro+ Shopping Channel         | Freyre              | Diego               | (91) 555 94 44   | C/ Moralzarzal, 86        | None             | 28034          | Madrid  | None      | Spain       | 227600.0        | 1370.0                    |


4.4. Create a list of the columns which you need to drop from the `df_orders` DataFrame - the ones associated with the `customers` table. You will keep the `customerNumber` field as it is the way to relate the two tables:

In [ ]:
customer_columns.pop(0)
customer_columns

4.5. Drop the columns specified in the list `customer_columns` from the `df_orders` DataFrame. You should keep `inplace` argument equal to `True`.

In [ ]:
### START CODE HERE ### (1 line of code)
df_orders.drop(columns=customer_columns, inplace=True) # @REPLACE None.None(columns=None, inplace=None)
### END CODE HERE ###

df_orders.head()

##### __Expected Output__

| **ordernumber** | **orderdate** | **requireddate** | **shippeddate** | **status** | **comments**             | **customerNumber** |
| --------------- | ------------- | ---------------- | ---------------- | ---------- | ----------------------- | ------------------ |
| 10100           | 2003-01-06    | 2003-01-13       | 2003-01-10       | Shipped    | None                     | 1                  |
| 10101           | 2003-01-09    | 2003-01-18       | 2003-01-11       | Shipped    | Check on availability.  | 2                  |
| 10102           | 2003-01-10    | 2003-01-18       | 2003-01-14       | Shipped    | None                     | 3                  |
| 10103           | 2003-01-29    | 2003-02-07       | 2003-02-02       | Shipped    | None                     | 4                  |
| 10104           | 2003-01-31    | 2003-02-09       | 2003-02-01       | Shipped    | None                     | 5                  |


Great! With those transformations you have achieved a 3NF from your initial OBT. 

4.6. Let's upload the data into the `classicmodels_3nf` schema. Remember that you have three tables: `customers`, `orders` and `orderdetails`. 

In [ ]:
%%sql
DROP TABLE IF EXISTS classicmodels_3nf.customers

In [ ]:
%%sql
DROP TABLE IF EXISTS classicmodels_3nf.orders

In [ ]:
%%sql
DROP TABLE IF EXISTS classicmodels_3nf.orderdetails

In [ ]:
df_customers.to_sql('customers', engine, schema='classicmodels_3nf', index=False)

In [ ]:
df_orders.to_sql('orders', engine, schema='classicmodels_3nf', index=False)

In [ ]:
df_orderdetails.to_sql('orderdetails', engine, schema='classicmodels_3nf', index=False)

Finally, you can take a look to each of the tables that have been stored in your database.

In [ ]:
%%sql
select count(*) from classicmodels_3nf.customers;

In [ ]:
%%sql
select count(*) from classicmodels_3nf.orders;

In [ ]:
%%sql
select count(*) from classicmodels_3nf.orderdetails;

In this lab, you have successfully transformed a One Big Table (OBT) into First Normal Form (1NF), Second Normal Form (2NF), and Third Normal Form (3NF). This process of normalization is crucial in designing efficient and reliable databases.

Throughout the lab, you have learned the importance of each normal form and the specific steps required to achieve them:

* 1NF: Ensuring that each table cell contains only atomic (indivisible) values and each record is unique.
* 2NF: Building on 1NF by removing partial dependencies, ensuring that non-key attributes are fully dependent on the composite primary key.
* 3NF: Further refining the table structure by removing transitive dependencies, ensuring that non-key attributes are dependent only on the primary key.

This lab has provided you with hands-on experience in transforming a dataset into a normalized form, highlighting the practical steps and considerations involved in database normalization. As you progress in your work as a Data Engineer, these skills will be invaluable in ensuring the quality and efficiency of the databases you design and maintain. 